In [50]:
import json
from tqdm import tqdm
from rtc_transit_equity.datasets import generate
data = generate()
print(data.keys())

Finished all dataset gathering and preprocessing in 2.347060203552246s
dict_keys(['route_df', 'ridership_df', 'tract_population_df', 'county_population_df', 'bus_stop_income', 'result'])


In [51]:
stops2ridership = {
    'VineyardRTA': "Woods Hole, Martha's Vineyard and Nantucket Steamship Authority",
    'CapeCodRTA': "Cape Cod Regional Transit Authority",
    'LowellRTA': 'Lowell Regional Transit Authority',
    'CapeAnnRTA': "Cape Ann Transportation Authority",
    'BerkshireRTA': "Berkshire Regional Transit Authority",
    'MontachusettRTA': "Montachusett Regional Transit Authority",
    'MerrimackValleyRTA': "Merrimack Valley Regional Transit Authority",
    'PioneerValleyRTA': 'Pioneer Valley Transit Authority',
    'MetroWestRTA': 'MetroWest Regional Transit Authority',
    'WRTA': 'Worcester Regional Transit Authority',
    'BrocktonAreaRTA': 'Brockton Area Transit Authority',
    'SoutheasternRTA': 'Southeastern Regional Transit Authority'
}

In [52]:
result = data['result']
ridership_df = data['ridership_df']
bus_stop_income = data['bus_stop_income']
routes = data['route_df']

# Strategic Q1
What bus routes and stops, if made free, would most benefit low income riders in Massachusetts?

To answer this question, we used 2 approaches. First, we found the average household income of all users within a route and analyzed routes with respect to the average median household income.

Next we will analyze routes with respect to the lowest median household income.


p = population census count of some bus stop

m = median household income of some bus stop

We defined the median household income of a route to be:
$$\frac{\sum p * m}{\sum p}$$

Where we are essentially normalizing the median household income of a route based on the stops along that route

In [53]:
route_ids = list(set(result.route_id))

# TODO FIGURE OUT WHY WE HAVE DUPLICATE AGENCIES
# Normalize route's median household income by bus stops along the route
averages = {}
for route_id in tqdm(route_ids):
    route_stops = result.loc[result.route_id == route_id]
    route_avg = (route_stops.population * route_stops.median_household_income).sum() / route_stops.population.sum()
    
    agencies = list(set(route_stops.Agency))
    averages[route_id] = {
        "median_household_income": float(route_avg), 
        "RTA": list(set(route_stops.Agency)),
        "route_population": float(route_stops.drop_duplicates('census_tract').population.sum()),
        "short_name": list(set(route_stops.route_short_name))
    }


100%|██████████| 221/221 [00:01<00:00, 140.60it/s]


In [54]:
vals = list(averages.items())
poorest_routes = sorted(vals, key=lambda elem: elem[1]["median_household_income"])
print(f"Poorest routes:\n\n{json.dumps(poorest_routes[:10], indent=2)}")

Poorest routes:

[
  [
    "B4",
    {
      "median_household_income": 16901.605198118472,
      "RTA": [
        "PioneerValleyRTA"
      ],
      "route_population": 8281.0,
      "short_name": [
        "B4"
      ]
    }
  ],
  [
    "921",
    {
      "median_household_income": 21671.941073823196,
      "RTA": [
        "PioneerValleyRTA"
      ],
      "route_population": 20286.0,
      "short_name": [
        "P21E"
      ]
    }
  ],
  [
    "3688",
    {
      "median_household_income": 26521.33830045623,
      "RTA": [
        "WRTA"
      ],
      "route_population": 11705.0,
      "short_name": [
        "6"
      ]
    }
  ],
  [
    "2995",
    {
      "median_household_income": 29320.265139403735,
      "RTA": [
        "LowellRTA"
      ],
      "route_population": 22544.0,
      "short_name": [
        "6"
      ]
    }
  ],
  [
    "2998",
    {
      "median_household_income": 30999.477108691663,
      "RTA": [
        "MerrimackValleyRTA",
        "LowellRTA"
     

In [55]:
vals = list(averages.items())
highest_population = sorted(vals, key=lambda elem: elem[1]["route_population"], reverse=True)
print(f"Highest population:\n\n{json.dumps(highest_population[:10], indent=2)}")

Highest population:

[
  [
    "3219",
    {
      "median_household_income": 87848.30053915674,
      "RTA": [
        "MerrimackValleyRTA",
        "BrocktonAreaRTA"
      ],
      "route_population": 120870.0,
      "short_name": [
        "12"
      ]
    }
  ],
  [
    "X90",
    {
      "median_household_income": 40835.40385290104,
      "RTA": [
        "PioneerValleyRTA"
      ],
      "route_population": 91807.0,
      "short_name": [
        "X90"
      ]
    }
  ],
  [
    "3487",
    {
      "median_household_income": 99274.3136949102,
      "RTA": [
        "GATRA_RTA"
      ],
      "route_population": 78781.0,
      "short_name": [
        "14"
      ]
    }
  ],
  [
    "G2",
    {
      "median_household_income": 46769.51143440453,
      "RTA": [
        "PioneerValleyRTA"
      ],
      "route_population": 70636.0,
      "short_name": [
        "G2"
      ]
    }
  ],
  [
    "3241",
    {
      "median_household_income": 79324.51225889487,
      "RTA": [
        "Cap

In [56]:
route_ids = list(set(result.route_id))

# TODO FIGURE OUT WHY WE HAVE DUPLICATE AGENCIES
# Normalize route's median household income by bus stops along the route
mins = {}
for route_id in tqdm(route_ids):
    route_stops = result.loc[result.route_id == route_id]
    min_income = route_stops.median_household_income.min()
    
    agencies = list(set(route_stops.Agency))
    mins[route_id] = {
        "min_median_household_income": float(min_income), 
        "RTA": list(set(route_stops.Agency)),
        "route_population": float(route_stops.drop_duplicates('census_tract').population.sum()),
        "short_name": list(set(route_stops.route_short_name))
    }


100%|██████████| 221/221 [00:01<00:00, 155.47it/s]


In [57]:
vals = list(mins.items())
min_income = sorted(vals, key=lambda elem: elem[1]["min_median_household_income"])
print(f"Min income:\n\n{json.dumps(min_income[:10], indent=2)}")

Min income:

[
  [
    "36",
    {
      "min_median_household_income": 2499.0,
      "RTA": [
        "PioneerValleyRTA"
      ],
      "route_population": 31772.0,
      "short_name": [
        "36"
      ]
    }
  ],
  [
    "B43",
    {
      "min_median_household_income": 2499.0,
      "RTA": [
        "FranklinRTA",
        "PioneerValleyRTA"
      ],
      "route_population": 40827.0,
      "short_name": [
        "B43"
      ]
    }
  ],
  [
    "30",
    {
      "min_median_household_income": 2499.0,
      "RTA": [
        "PioneerValleyRTA"
      ],
      "route_population": 31155.0,
      "short_name": [
        "30"
      ]
    }
  ],
  [
    "33",
    {
      "min_median_household_income": 2499.0,
      "RTA": [
        "FranklinRTA",
        "PioneerValleyRTA"
      ],
      "route_population": 46405.0,
      "short_name": [
        "33"
      ]
    }
  ],
  [
    "2907",
    {
      "min_median_household_income": 13845.0,
      "RTA": [
        "SoutheasternRTA"
      ],

# Strategic Q2

C: Total Operating Cost

T: Total number of unlinked trips

F: Total fares in a fiscal year

Average Cost = $$\frac{C}{T}$$

Free Estimated Average Cost Per Trip = $$\frac{C + F}{T * 1.3}$$

We use the 1.3 constant due to the simpson curtson rule

In [58]:
RIDERSHIP_INCREASE_CONSTANT = 1.3

ridership_df['Free Estimated Average Cost Per Trip'] = \
    (ridership_df['Operating Expenses FY'] + ridership_df['Fares FY']) / \
    (ridership_df['Unlinked Passenger Trips FY'] * RIDERSHIP_INCREASE_CONSTANT)

ridership_df['Free percent average cost change'] = \
    (ridership_df['Free Estimated Average Cost Per Trip'] - ridership_df['Average Cost per Trip FY']) / \
    ridership_df['Average Cost per Trip FY']

In [59]:
display(ridership_df[[
    'Free Estimated Average Cost Per Trip',
    'Average Cost per Trip FY',
    'Free percent average cost change',
    'Agency'
]].sort_values('Free percent average cost change').dropna())

,Free Estimated Average Cost Per Trip,Average Cost per Trip FY,Free percent average cost change,Agency
17,5.843595,7.5967,-0.230772,"Woods Hole, Martha's Vineyard and Nantucket St..."
16,38.680183,48.8760,-0.208606,Worcester Regional Transit Authority COA
6,6.272620,7.5696,-0.171341,Merrimack Valley Regional Transit Authority
9,7.516318,9.0034,-0.165169,Cape Ann Transportation Authority
11,9.408694,11.1007,-0.152423,Greater Attleboro-Taunton Regional Transit Aut...
15,7.948441,9.1898,-0.135080,MetroWest Regional Transit Authority
10,9.334594,10.7798,-0.134066,Montachusett Regional Transit Authority
4,9.223292,10.6482,-0.133817,Berkshire Regional Transit Authority
2,6.260449,7.1723,-0.127135,Lowell Regional Transit Authority
13,9.986896,11.3875,-0.122995,Cape Cod Regional Transit Authority


Below I will prove that assuming we have a 30% increase of ridership when we remove 100% of fares via the simpson curtson rule, average cost per trip will always decrease when the amount of revenue you generate is less than 3 tenths of the total operational cost:

I will define the below function to be the percent change of average cost per trip when you remove 100% of fares. Let's define the following variables:

x: Total revenue from fares

C: Total cost of operation

T: Total number of trips

$$g(x) = \frac{(\frac{c + x}{t*1.3}) - \frac{c}{t}}{\frac{c}{t}}$$

Notice this function will be strictly increasing when C and T are positive numbers.

Now to solve for when y=0:

$$0 = \frac{(\frac{c + x}{t*1.3}) - \frac{c}{t}}{\frac{c}{t}}$$

$$0 = \frac{c + x}{t*1.3} - \frac{c}{t}$$

$$\frac{c}{t} = \frac{c + x}{t*1.3}$$

$$c * 1.3 = c + x$$

$$c * 1.3 - c = x$$

$$.3 * c = x$$

data['result']

In [60]:
ridership_df

,5 digit NTD ID,Agency,Service Area Population,TOS,Active,Passenger Miles FY,Unlinked Passenger Trips FY,Fares FY,Operating Expenses FY,Average Cost per Trip FY,Average Fares per Trip FY,Free Estimated Average Cost Per Trip,Free percent average cost change
0,10004,Brockton Area Transit Authority,254648.0,PT,Active,17731983.0,2636726.0,2668996.0,11706738.0,4.4399,1.0122,4.193935,-0.055399
1,10005,Lowell Regional Transit Authority,338186.0,DO,Active,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,10005,Lowell Regional Transit Authority,338186.0,PT,Active,6620433.0,1370690.0,1324536.0,9830939.0,7.1723,0.9663,6.260449,-0.127135
3,10006,Southeastern Regional Transit Authority,308614.0,PT,Active,8586355.0,2666570.0,2267445.0,14368729.0,5.3885,0.8503,4.799070,-0.109387
4,10007,Berkshire Regional Transit Authority,127500.0,PT,Active,4477482.0,497498.0,667672.0,5297468.0,10.6482,1.3421,9.223292,-0.133817
5,10008,Pioneer Valley Transit Authority,551543.0,PT,Active,39422834.0,10120344.0,6732600.0,39216072.0,3.8750,0.6653,3.492483,-0.098714
6,10013,Merrimack Valley Regional Transit Authority,306339.0,PT,Active,8095787.0,1952888.0,1141975.0,14782667.0,7.5696,0.5848,6.272620,-0.171341
7,10014,Worcester Regional Transit Authority,479329.0,DO,Active,13078228.0,3013268.0,2847838.0,20250045.0,6.7203,0.9451,5.896456,-0.122590
8,10014,Worcester Regional Transit Authority,479329.0,PT,Active,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,10053,Cape Ann Transportation Authority,54099.0,PT,Active,790537.0,206000.0,158172.0,1854698.0,9.0034,0.7678,7.516318,-0.165169


# Strategic Question 3

What would the cost be to the MBTA and regional transit authorities for each proposed bus route/stop/zones based on ridership and fare costs?

There is no ridership data per bus stop in MA available, so in order to answer this, we must first generate our own ridership data per bus stop.

Our data currently only has ridership per RTA, and we are able to map bus stops onto RTA's. We can use this generalized ridership data in combination with the bus stops population density to draw estimates, however truthfully these estimates will be far more inaccurate than if we had ridership per bus stop basis.

After merging our ridership data onto our bus stop income, we will define the cost of a bus stop as the difference in fare revenue if that bus stop were to be removed. 

In [61]:
# Uniform Agency names across datasets
bus_stop_income['Agency'] = bus_stop_income['Agency'].apply(
    lambda agency: stops2ridership[agency] if agency in stops2ridership else None
)

routes['Agency'] = routes['Agency'].apply(
    lambda agency: stops2ridership[agency] if agency in stops2ridership else None
)

# Add ridership data estimations to bus stops
bus_stop_ridership = bus_stop_income.merge(
    ridership_df, 
    on='Agency', 
)

# normalize fares with respect to population at this stop
bus_stop_ridership['Bus Stop Estimated Fares'] = bus_stop_ridership.apply(
    lambda row: (row['Fares FY'] / row['Service Area Population']) * row.population,
    axis=1
)

bus_stop_ridership[['Bus Stop Estimated Fares', 
                    'stop_name', 
                    'Agency', 
                    'TOS']]

,Bus Stop Estimated Fares,stop_name,Agency,TOS
0,62415.063853,Market Basket,Brockton Area Transit Authority,PT
1,62415.063853,Belair St at Colonel Bell Drive,Brockton Area Transit Authority,PT
2,62415.063853,Belair St at Belair Hi Rise,Brockton Area Transit Authority,PT
3,62415.063853,Belair St at Belair Hi Rise,Brockton Area Transit Authority,PT
4,62415.063853,Belair St at Earle St,Brockton Area Transit Authority,PT
...,...,...,...,...
9788,12469.341208,Coolidge at Sudbury,MetroWest Regional Transit Authority,PT
9789,10871.243365,Wayland Center,MetroWest Regional Transit Authority,PT
9790,10871.243365,Coach Grill,MetroWest Regional Transit Authority,PT
9791,10981.961255,Colonial Rd Capital Rd Housing,MetroWest Regional Transit Authority,PT


Now we will try and analyze our ridership data on a per route basis as seen below:

In [62]:
routes_ridership = routes.merge(ridership_df, on='Agency')
routes_ridership['population'] = routes_ridership.apply(
    lambda row: averages[row.route_id]['route_population'],
    axis=1
)

routes_ridership['Routes Estimated Fares'] = routes_ridership.apply(
    lambda row: (row['Fares FY'] / row['Service Area Population']) * row.population,
    axis=1
)
cost_impact_per_route = routes_ridership[['Routes Estimated Fares', 
                    'route_long_name',
                    'route_short_name',
                    'Agency', 
                    'TOS']]

In [67]:
cost_impact_per_route

,Routes Estimated Fares,route_long_name,route_short_name,Agency,TOS
0,214967.751406,Montello Street Via North Main Street,1,Brockton Area Transit Authority,PT
1,332450.614668,S Plaza/Campello Via Main Street,2,Brockton Area Transit Authority,PT
2,309088.200339,VA Hospital via Belmont,3,Brockton Area Transit Authority,PT
3,309088.200339,VA Hospital via Belmont,3,Brockton Area Transit Authority,PT
4,521886.360105,Southfield via Warren & Plain Street,8,Brockton Area Transit Authority,PT
...,...,...,...,...,...
1193,18188.654356,Yellow Line,NaN,Cape Ann Transportation Authority,PT
1194,18188.654356,Yellow Line,NaN,Cape Ann Transportation Authority,PT
1195,62015.680364,Purple Line,NaN,Cape Ann Transportation Authority,PT
1196,189327.489602,City of Beverly Shuttle (Yellow),NaN,Cape Ann Transportation Authority,PT


# Strategic Question 4

What would the cost be to make an entire regional transit area free and how would this compare?

For RTA's this difference in Fares of which we define as the cost to the city is much easier and more accurate to analyze because we just need to look at MA's ridership data per RTA as seen below:

In [64]:
ridership_df[['Agency', 'Fares FY', 'TOS']]

,Agency,Fares FY,TOS
0,Brockton Area Transit Authority,2668996.0,PT
1,Lowell Regional Transit Authority,NaN,DO
2,Lowell Regional Transit Authority,1324536.0,PT
3,Southeastern Regional Transit Authority,2267445.0,PT
4,Berkshire Regional Transit Authority,667672.0,PT
5,Pioneer Valley Transit Authority,6732600.0,PT
6,Merrimack Valley Regional Transit Authority,1141975.0,PT
7,Worcester Regional Transit Authority,2847838.0,DO
8,Worcester Regional Transit Authority,NaN,PT
9,Cape Ann Transportation Authority,158172.0,PT
